# importing the MLBox

In [1]:
from mlbox.preprocessing import *
from mlbox.optimisation import *
from mlbox.prediction import *

/home/datageek/anaconda2/lib/python2.7/site-packages/mlbox/preprocessing/drift/__init__.py:9: UserWarning: ipCluster is starting. Please wait 30 sec and check in terminal that 'the engines appear to have started successfully'.
  warnings.warn("ipCluster is starting. Please wait 30 sec and check in terminal that 'the engines appear to have started successfully'.")
Using Theano backend.
/home/datageek/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [13]:
paths = ["train.csv","test.csv"]
target_name="SalePrice"

# reading and cleaning all files 

In [3]:
rd = Reader(sep = ',')
df = rd.train_test_split(paths, target_name)


reading csv : train.csv ...
cleaning data...
CPU time: 19.9882261753 seconds

reading csv : test.csv ...
cleaning data...
CPU time: 0.484149932861 seconds

number of common features : 80

gathering and crunching for train and test datasets
reindexing for train and test datasets
dropping training duplicates
dropping constant variables on training set

number of categorical features : 43
number of numerical features : 37
number of training samples : 1460
number of test samples : 1459

task : regression


In [4]:
dft = Drift_thresholder()
df = dft.fit_transform(df)


computing drifts...
CPU time: 1.19959688187 seconds

Top 10 drifts

('Id', 0.9996570644718793)
('GarageYrBlt', 0.52930157759352303)
('Exterior1st', 0.52586411001119482)
('TotalBsmtSF', 0.52389869237891595)
('2ndFlrSF', 0.52123329243075833)
('Exterior2nd', 0.52107292808122652)
('BsmtFullBath', 0.52018890605818879)
('GrLivArea', 0.5197186371334116)
('TotRmsAbvGrd', 0.5194699326175769)
('WoodDeckSF', 0.51865392689173118)

deleted variables : ['Id']

dumping drift coefficients into directory : save
drift coefficients dumped


# tuning

In [28]:
mape = make_scorer(lambda y_true, y_pred: 100*np.sum(np.abs(y_true-y_pred)/y_true)/len(y_true), greater_is_better=False, needs_proba=False)
opt = Optimiser(scoring = mape, n_folds = 3)

In [27]:
opt.evaluate(None, df)

No parameters set. Default configuration is tested

########################################################## testing hyper-parameters... #################################################################

>>> NA ENCODER :{'numerical_strategy': 'mean', 'categorical_strategy': '<NULL>'}

>>> CA ENCODER :{'strategy': 'label_encoding'}

>>> ESTIMATOR :{'seed': 0, 'min_child_weight': 5, 'max_drop': 50, 'min_split_gain': 0, 'boosting_type': 'gbdt', 'silent': True, 'min_child_samples': 10, 'nthread': -1, 'fair_c': 1.0, 'strategy': 'LightGBM', 'max_bin': 255, 'uniform_drop': False, 'max_depth': -1, 'skip_drop': 0.5, 'num_leaves': 31, 'reg_alpha': 0, 'subsample_for_bin': 50000, 'learning_rate': 0.05, 'gaussian_eta': 1.0, 'n_estimators': 500, 'objective': 'regression', 'huber_delta': 1.0, 'poisson_max_delta_step': 0.7, 'colsample_bytree': 0.8, 'subsample_freq': 1, 'xgboost_dart_mode': False, 'subsample': 0.9, 'reg_lambda': 0, 'drop_rate': 0.1}


MEAN SCORE : make_scorer(<lambda>, greater_is_bet

-16.528187887133559

In [29]:
space = {
    
        'ne__numerical_strategy':{"search":"choice",
                                 "space":[0]},
        'ce__strategy':{"search":"choice",
                        "space":["label_encoding","random_projection", "entity_embedding"]}, 
        'fs__threshold':{"search":"uniform",
                        "space":[0.01,0.3]},    
        'est__max_depth':{"search":"choice",
                                  "space":[3,4,5,6,7]}
    
        }

best = opt.optimise(space, df,15)


########################################################## testing hyper-parameters... #################################################################

>>> NA ENCODER :{'numerical_strategy': 0, 'categorical_strategy': '<NULL>'}

>>> CA ENCODER :{'strategy': 'label_encoding'}

>>> FEATURE SELECTOR :{'threshold': 0.17310745571388583, 'strategy': 'l1'}

>>> ESTIMATOR :{'seed': 0, 'min_child_weight': 5, 'max_drop': 50, 'min_split_gain': 0, 'boosting_type': 'gbdt', 'silent': True, 'min_child_samples': 10, 'nthread': -1, 'fair_c': 1.0, 'strategy': 'LightGBM', 'max_bin': 255, 'uniform_drop': False, 'max_depth': 6, 'skip_drop': 0.5, 'num_leaves': 31, 'reg_alpha': 0, 'subsample_for_bin': 50000, 'learning_rate': 0.05, 'gaussian_eta': 1.0, 'n_estimators': 500, 'objective': 'regression', 'huber_delta': 1.0, 'poisson_max_delta_step': 0.7, 'colsample_bytree': 0.8, 'subsample_freq': 1, 'xgboost_dart_mode': False, 'subsample': 0.9, 'reg_lambda': 0, 'drop_rate': 0.1}


MEAN SCORE : make_scorer(<lamb


MEAN SCORE : make_scorer(<lambda>, greater_is_better=False) = -17.2975569155
VARIANCE : 2.06550076276 (fold 1 = -20.1336601, fold 2 = -16.4851695423, fold 3 = -15.2738411041)fold 3 = -15.2738411041)
CPU time: 2.79752588272 seconds


########################################################## testing hyper-parameters... #################################################################

>>> NA ENCODER :{'numerical_strategy': 0, 'categorical_strategy': '<NULL>'}

>>> CA ENCODER :{'strategy': 'random_projection'}

>>> FEATURE SELECTOR :{'threshold': 0.21366855829704756, 'strategy': 'l1'}

>>> ESTIMATOR :{'seed': 0, 'min_child_weight': 5, 'max_drop': 50, 'min_split_gain': 0, 'boosting_type': 'gbdt', 'silent': True, 'min_child_samples': 10, 'nthread': -1, 'fair_c': 1.0, 'strategy': 'LightGBM', 'max_bin': 255, 'uniform_drop': False, 'max_depth': 7, 'skip_drop': 0.5, 'num_leaves': 31, 'reg_alpha': 0, 'subsample_for_bin': 50000, 'learning_rate': 0.05, 'gaussian_eta': 1.0, 'n_estimators': 500, '


MEAN SCORE : make_scorer(<lambda>, greater_is_better=False) = -16.5616289975
VARIANCE : 1.93349554079 (fold 1 = -19.2634463172, fold 2 = -15.575062056, fold 3 = -14.8463786194)fold 3 = -14.8463786194)
CPU time: 7.67966794968 seconds


########################################################## testing hyper-parameters... #################################################################

>>> NA ENCODER :{'numerical_strategy': 0, 'categorical_strategy': '<NULL>'}

>>> CA ENCODER :{'strategy': 'random_projection'}

>>> FEATURE SELECTOR :{'threshold': 0.2552044856956169, 'strategy': 'l1'}

>>> ESTIMATOR :{'seed': 0, 'min_child_weight': 5, 'max_drop': 50, 'min_split_gain': 0, 'boosting_type': 'gbdt', 'silent': True, 'min_child_samples': 10, 'nthread': -1, 'fair_c': 1.0, 'strategy': 'LightGBM', 'max_bin': 255, 'uniform_drop': False, 'max_depth': 5, 'skip_drop': 0.5, 'num_leaves': 31, 'reg_alpha': 0, 'subsample_for_bin': 50000, 'learning_rate': 0.05, 'gaussian_eta': 1.0, 'n_estimators': 500, 

In [30]:
prd = Predictor()
prd.fit_predict(best,df)


fitting the pipeline...
pipeline dumped into directory : save/joblib
CPU time: 2.76346993446 seconds

predicting...
CPU time: 0.514960050583 seconds

top 10 predictions :

   SalePrice_predicted
0        133596.832674
1        143311.494308
2        172966.573265
3        178759.592986
4        201374.579311
5        177493.158927
6        167828.486093
7        170111.871096
8        180243.803278
9        132210.439378

dumping predictions into directory : save


<mlbox.prediction.predictor.Predictor instance at 0x7f8348cddfc8>